데이터에서 결측치 개수 확인

In [1]:
import pandas as pd

from mymodules.paths import Paths

In [2]:
# 각 컬럼별 결측치 개수
def check_null(filename: str) -> tuple[int, pd.Series, map]:
    columns = pd.read_csv(filename, index_col='Test_id', nrows=10).columns
    null_rows = {col: [] for col in columns}

    total_rows = 0
    total_nulls = pd.Series(
        0,
        index=columns,
        dtype='int64'
    )# total_nulls 초기화 (모든 컬럼에 대해 0)

    reader = pd.read_csv(filename, chunksize=10000)
    for _, df in enumerate(reader):
        total_rows += len(df)

        ns = df.isnull().sum()
        total_nulls += ns

        if ns.sum() > 0: # 결측 있음.
            for col, cnt in ns[ns > 0].items():
                null_idx = df.index[df[col].isnull()].tolist()
                if null_idx:
                    null_rows[col] += [i for i in null_idx]

    return total_rows, total_nulls, null_rows

def print_null(filename: str) -> None:
    total_rows, total_nulls, null_rows = check_null(filename)

    print('------------------------------', filename)
    print('총 행 수:', total_rows)
    print('컬럼별 총 결측 개수:')
    print(total_nulls)
    print('-----')
    print('컬럼별 결측치 위치:')
    agg = []
    for k, v in null_rows.items():
        if v:
            print(f'{k}: {v}')
            agg += v
    agg_l = list(set(agg))
    agg_l.sort()
    print('모든 결측치 위치: ', agg_l)

In [3]:
print_null(Paths('open_v2').train_A)

------------------------------ open_v2\data\train\A.csv
총 행 수: 647241
컬럼별 총 결측 개수:
Test          0.0
PrimaryKey    0.0
Age           0.0
TestDate      0.0
A1-1          0.0
A1-2          0.0
A1-3          0.0
A1-4          0.0
A2-1          1.0
A2-2          1.0
A2-3          1.0
A2-4          1.0
A3-1          1.0
A3-2          1.0
A3-3          1.0
A3-4          1.0
A3-5          1.0
A3-6          1.0
A3-7          1.0
A4-1          0.0
A4-2          0.0
A4-3          0.0
A4-4          0.0
A4-5          0.0
A5-1          2.0
A5-2          2.0
A5-3          2.0
A6-1          0.0
A7-1          0.0
A8-1          0.0
A8-2          0.0
A9-1          0.0
A9-2          0.0
A9-3          0.0
A9-4          0.0
A9-5          0.0
dtype: float64
-----
컬럼별 결측치 위치:
A2-1: [526696]
A2-2: [526696]
A2-3: [526696]
A2-4: [526696]
A3-1: [101788]
A3-2: [101788]
A3-3: [101788]
A3-4: [101788]
A3-5: [101788]
A3-6: [101788]
A3-7: [101788]
A5-1: [49543, 448070]
A5-2: [49543, 448070]
A5-3: [49543, 448070]
모든 결측

In [4]:
print_null(Paths('open_v2').train_B)

------------------------------ open_v2\data\train\B.csv
총 행 수: 297526
컬럼별 총 결측 개수:
Test          0.0
PrimaryKey    0.0
Age           0.0
TestDate      0.0
B1-1          9.0
B1-2          9.0
B1-3          9.0
B2-1          3.0
B2-2          3.0
B2-3          3.0
B3-1          3.0
B3-2          3.0
B4-1          3.0
B4-2          3.0
B5-1          3.0
B5-2          3.0
B6            3.0
B7            2.0
B8            1.0
B9-1          0.0
B9-2          0.0
B9-3          0.0
B9-4          0.0
B9-5          0.0
B10-1         0.0
B10-2         0.0
B10-3         0.0
B10-4         0.0
B10-5         0.0
B10-6         0.0
dtype: float64
-----
컬럼별 결측치 위치:
B1-1: [19828, 19856, 20141, 20858, 21052, 180529, 216541, 221682, 288862]
B1-2: [19828, 19856, 20141, 20858, 21052, 180529, 216541, 221682, 288862]
B1-3: [19828, 19856, 20141, 20858, 21052, 180529, 216541, 221682, 288862]
B2-1: [180529, 221682, 288862]
B2-2: [180529, 221682, 288862]
B2-3: [180529, 221682, 288862]
B3-1: [180529, 221682, 288862

In [5]:
# row 한 개만 읽기
def get_one_row(filename:str, row_number: int) -> pd.DataFrame:
    header = pd.read_csv(filename, nrows=0)# 헤더(컬럼명)
    row = pd.read_csv(filename, skiprows=row_number+1, nrows=1, header=None)# 특정 행만 읽기 (header 없음)
    row.columns = header.columns# 헤더 붙이기
    return row

get_one_row(Paths('open_v2').train_B, 19828)

,Test_id,Test,PrimaryKey,Age,TestDate,B1-1,B1-2,B1-3,B2-1,B2-2,...,B9-2,B9-3,B9-4,B9-5,B10-1,B10-2,B10-3,B10-4,B10-5,B10-6
0,0x1E8A522C1DC242570EB3F04102FCA5807297EC637BAF...,B,0x1E8A522C1DC242570EB3F04102FCA5807297EC637BAF...,70a,201901,NaN,NaN,NaN,"2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0",...,0,0,35,6,17,3,0,60,2,19
